In [360]:
import os
import numpy
import pandas as pd
import sqlalchemy as sqlal
import pymongo
import json
import datetime
import certifi
import sqlalchemy
from sqlalchemy import create_engine

In [361]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Passw0rd123"

src_dbname = "Chinook"
dst_dbname = "Chinook_dw"

In [362]:
#From lab 3
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection)
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(sqlal.text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


In [363]:
#From lab 4
atlas_cluster_name = "cluster0.6kd78gj"
atlas_user_name = "rmg7jb"
atlas_password = "Passw0rd123"

mongo_str = f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net?retryWrites=true&w=majority&appName=Cluster0"

def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe
client = pymongo.MongoClient(mongo_str, tlsCAFile=certifi.where())
db = client[src_dbname]

# Gets the path of the Current Working Directory for this Notebook, and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd())

json_files = {"genre" : 'genre.json'}

for file in json_files:
    db.drop_collection(file)
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)
        #print(f"{file} was successfully loaded.")
client.close()        

In [364]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
conn = sqlEngine.connect()
conn.execute(sqlal.text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
conn.execute(sqlal.text(f"CREATE DATABASE `{dst_dbname}`;"))
conn.execute(sqlal.text(f"USE {dst_dbname};"))
conn.close()

In [365]:
#Create dimension tables in dw
sql_customers = sqlal.text("SELECT * FROM Chinook.Customer;")
df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
df_customers.head(2)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [366]:
#Check to see if needs to be cleaned
for column in df_customers.columns:
    print(str(column),str(df_customers[column].isnull().sum()))
#All looks good. Vars with null values are meaningful (ex: states null when customer not from U.S.)
df_customers.rename(columns={"CustomerId":"customer_id"}, inplace=True)
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))

CustomerId 0
FirstName 0
LastName 0
Company 49
Address 0
City 0
State 29
Country 0
PostalCode 4
Phone 1
Fax 47
Email 0
SupportRepId 0


In [367]:
sql_tracks = sqlal.text("SELECT * FROM Chinook.Track;")
df_tracks = get_dataframe(user_id, pwd, host_name, src_dbname, sql_tracks)
df_tracks.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,"U. Dirkschneider, W. Hoffmann, H. Frank, P. Ba...",342562,5510424,0.99


In [368]:
#Check to see if needs to be cleaned
for column in df_tracks.columns:
    print(str(column),str(df_tracks[column].isnull().sum()))
#Composer only var with nulls. We will keep it.
df_tracks.rename(columns={"TrackId":"track_id",'GenreId':'genre_id','AlbumId':'album_id','MediaTypeId':'media_type_id'}, inplace=True)
df_tracks.insert(0, "track_key", range(1, df_tracks.shape[0]+1))

TrackId 0
Name 0
AlbumId 0
MediaTypeId 0
GenreId 0
Composer 977
Milliseconds 0
Bytes 0
UnitPrice 0


In [369]:
#Let's pull JSON data from MongoDB.
query = {} # Select all elements (columns), and all documents (rows).
collection = "genre"
df_genre = get_mongo_dataframe(mongo_str, src_dbname, collection, query)
df_genre.head(2)

,GenreId,Name
0,1,Rock
1,2,Jazz


In [370]:
#Check to see if needs to be cleaned
for column in df_genre.columns:
    print(str(column),str(df_genre[column].isnull().sum()))
#Everything is good
df_genre.rename(columns={"GenreId":"genre_id"}, inplace=True)
df_genre.insert(0, "genre_key", range(1, df_genre.shape[0]+1))

GenreId 0
Name 0


In [371]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_tracks', df_tracks, 'track_key'),
          ('dim_genre',df_genre,'genre_key')]
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [372]:
#Create fact table in dw
#First, see if we can pull invoice table out
sql_invoices = sqlal.text("SELECT * FROM Chinook.Invoice;")
df_invoices = get_dataframe(user_id,pwd,host_name,src_dbname,sql_invoices)
#Change column names to make them easier to work with
df_invoices.rename(columns={"InvoiceId":"invoice_id","CustomerId":"customer_id","InvoiceDate":"invoice_date","BillingAddress":"address","BillingCity":"billing_city","BillingCountry":"billing_country","BillingPostalCode":"billing_zip"},inplace=True)
#Drop customer info since we can pull that from dim_customers if needed
fact_table = df_invoices
fact_table.drop(['address','billing_city','BillingState','billing_country','billing_zip'],axis=1,inplace=True)
fact_table.head()


,invoice_id,customer_id,invoice_date,Total
0,1,2,2021-01-01,1.98
1,2,4,2021-01-02,3.96
2,3,8,2021-01-03,5.94
3,4,14,2021-01-06,8.91
4,5,23,2021-01-11,13.86


In [373]:
#Pull data from local
local_file = "invoiceline.csv"
#MySQLWorkbench delineates using ,. We need ;
invoiceline_df = pd.read_csv(local_file,sep=';')
invoiceline_df

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
...,...,...,...,...,...
2235,2236,411,3136,0.99,1
2236,2237,411,3145,0.99,1
2237,2238,411,3154,0.99,1
2238,2239,411,3163,0.99,1


In [374]:
invoiceline_df.rename(columns={"InvoiceId":"invoice_id","InvoiceLineId":"invoice_line_id","UnitPrice":"unit_price",'TrackId':'track_id'},inplace=True)
invoiceline_df

,invoice_line_id,invoice_id,track_id,unit_price,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1
...,...,...,...,...,...
2235,2236,411,3136,0.99,1
2236,2237,411,3145,0.99,1
2237,2238,411,3154,0.99,1
2238,2239,411,3163,0.99,1


In [375]:
fact_table = pd.merge(fact_table,invoiceline_df,on='invoice_id')
fact_table.rename(columns={'Quantity':'quantity'},inplace=True)

In [376]:
ordered_columns = ['invoice_id','invoice_line_id','track_id','customer_id','invoice_date','unit_price','quantity']
fact_table = fact_table[ordered_columns]

In [377]:
fact_table.insert(0, "id", range(1, fact_table.shape[0]+1))

In [378]:
fact_table['invoice_date'] = fact_table['invoice_date'].astype('datetime64[ns]')

In [380]:
#Need invoice_date to be in correct date_key format. Also needed to edit SQL query given, since this data goes beyond 2010.
sql_date = sqlal.text("SELECT `date_key`,`full_date` FROM Chinook_dw.dim_date;")
dw_dbname = "Chinook_dw"
df_date = get_dataframe(user_id, pwd, host_name, dw_dbname, sql_date)
df_date.full_date = df_date.full_date.astype('datetime64[ns]')
df_date.tail()

,date_key,full_date
10588,20281227,2028-12-27
10589,20281228,2028-12-28
10590,20281229,2028-12-29
10591,20281230,2028-12-30
10592,20281231,2028-12-31


In [381]:
date_mapping = df_date.set_index('full_date')['date_key'].to_dict()
fact_table['invoice_date'] = fact_table['invoice_date'].map(date_mapping)
fact_table

,id,invoice_id,invoice_line_id,track_id,customer_id,invoice_date,unit_price,quantity
0,1,1,1,2,2,20210101,0.99,1
1,2,1,2,4,2,20210101,0.99,1
2,3,2,3,6,4,20210102,0.99,1
3,4,2,4,8,4,20210102,0.99,1
4,5,2,5,10,4,20210102,0.99,1
...,...,...,...,...,...,...,...,...
2235,2236,411,2236,3136,44,20251214,0.99,1
2236,2237,411,2237,3145,44,20251214,0.99,1
2237,2238,411,2238,3154,44,20251214,0.99,1
2238,2239,411,2239,3163,44,20251214,0.99,1


In [383]:
db_operation = "insert"
tables = [('fact_table', fact_table, 'id')]
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [411]:
#SQL query to return the name and quantity sold of the top 5 selling tracks in the US
sql_query = sqlal.text("""
SELECT Name, SUM(quantity) as total_quantity, Country
FROM Chinook_dw.dim_tracks
JOIN Chinook_dw.fact_table ON Chinook_dw.dim_tracks.track_id = Chinook_dw.fact_table.track_id
JOIN Chinook_dw.dim_customers ON Chinook_dw.fact_table.customer_id = Chinook_dw.dim_customers.customer_id
WHERE dim_customers.Country = 'USA'
GROUP BY dim_tracks.Name ORDER BY total_quantity DESC LIMIT 5;
"""
)
result =  get_dataframe(user_id, pwd, host_name, src_dbname, sql_query)
result

,Name,total_quantity,Country
0,Heaven Can Wait,3.0,USA
1,Branch Closing,2.0,USA
2,Gay Witch Hunt,2.0,USA
3,Peace On Earth,2.0,USA
4,Música No Ar,2.0,USA
